# VacationPy
----


In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
# File to Load:
file_to_load='../weatherpy/WeatherPy.csv' # csv file created in part I of homework 

# Read csv WeatherPy file and store into data frame:
WeatherPy_df=pd.read_csv(file_to_load)
WeatherPy_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Jamestown,42.0970,-79.2353,65.52,36,20.0,5.75,US,1.649802e+09
1,Barrow,71.2906,-156.7887,1.42,66,0.0,9.22,US,1.649802e+09
2,Butaritari,3.0707,172.7902,81.21,78,44.0,16.46,KI,1.649801e+09
3,Rikitea,-23.1203,-134.9692,79.18,69,5.0,16.71,PF,1.649802e+09
4,Saint-Augustin,51.2260,-58.6502,33.31,98,87.0,5.01,CA,1.649802e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Determine max humidity level in the WeatherPy data frame:
max_hum=WeatherPy_df['Humidity'].max()


# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
WeatherPy_df = WeatherPy_df.dropna()
humidity = WeatherPy_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = WeatherPy_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=max_hum,
                               point_radius = 4)
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# Create new data frame with requested weather conditions:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows with null values

filtered_weather_df=WeatherPy_df[(WeatherPy_df['Max Temp'] < 80) &
                             (WeatherPy_df['Max Temp'] > 70) &
                             (WeatherPy_df['Wind Speed'] < 10) &
                             (WeatherPy_df['Cloudiness'] == 0)]

filtered_weather_df=filtered_weather_df.dropna() # drop any rows will null values

filtered_weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,Atlantic City,39.3643,-74.4229,73.58,38,0.0,8.05,US,1.649802e+09
182,Mossamedes,-15.1961,12.1522,75.33,77,0.0,4.59,AO,1.649802e+09
184,Gwadar,25.1216,62.3254,78.91,80,0.0,9.73,PK,1.649802e+09
408,Maceió,-9.6658,-35.7353,76.44,88,0.0,4.61,BR,1.649802e+09
452,Māngrol,21.1167,70.1167,79.45,77,0.0,9.40,IN,1.649802e+09
518,Vallenar,-28.5708,-70.7581,71.15,41,0.0,5.37,CL,1.649802e+09


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print('-------------------------------')
    


Retrieving Results for Index 32: Atlantic City.
Closest hotel is Courtyard by Marriott Atlantic City Beach Block.
-------------------------------
Retrieving Results for Index 182: Mossamedes.
Closest hotel is Hotel Chik Chik Namibe.
-------------------------------
Retrieving Results for Index 184: Gwadar.
Closest hotel is Sadaf Resort.
-------------------------------
Retrieving Results for Index 408: Maceió.
Closest hotel is Hotel Ponta Verde Maceió.
-------------------------------
Retrieving Results for Index 452: Māngrol.
Closest hotel is Hotel The Royal Honours.
-------------------------------
Retrieving Results for Index 518: Vallenar.
Closest hotel is Humacao Bed & Breakfast.
-------------------------------


In [13]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
32,Atlantic City,39.3643,-74.4229,73.58,38,0.0,8.05,US,1.649802e+09,Courtyard by Marriott Atlantic City Beach Block
182,Mossamedes,-15.1961,12.1522,75.33,77,0.0,4.59,AO,1.649802e+09,Hotel Chik Chik Namibe
184,Gwadar,25.1216,62.3254,78.91,80,0.0,9.73,PK,1.649802e+09,Sadaf Resort
408,Maceió,-9.6658,-35.7353,76.44,88,0.0,4.61,BR,1.649802e+09,Hotel Ponta Verde Maceió
452,Māngrol,21.1167,70.1167,79.45,77,0.0,9.40,IN,1.649802e+09,Hotel The Royal Honours
518,Vallenar,-28.5708,-70.7581,71.15,41,0.0,5.37,CL,1.649802e+09,Humacao Bed & Breakfast


In [14]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Customize the size of the figure:
# Add marker layer ontop of heat map
# Display figure

# Customize the size of the figure:
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Assign the marker layer to a variable:
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)


# Display figure
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig




Figure(layout=FigureLayout(height='420px'))